# Bài toán

> **Xử lý dữ liệu sau khi thu thập báo Việt Nam (Demo + Exercise)**

> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```


Vấn đề:

- Dữ liệu thu thập về có thể vẫn có nhiều vấn đề do lỗi khi thu thập và lỗi của bài báo (description ngắn, content ngắn, nội dung toàn ảnh, ..).
- Đồng nhất (Normalizing) giữa các trang web với nhau đưa về một dạng dữ liệu chung.

Mục tiêu:

- Xử lý từng dữ liệu của mỗi trang web (loại bỏ các bài báo không hợp lệ, đi qua từng trường dữ liệu và xử lý cơ bản)
- Hợp nhất dữ liệu các trang web với nhau

Đầu ra:

-  Một tập file JSON chứa các bài bài báo có các trường dữ liệu (Nhìn chung giống cái đã thu thập về và bổ sung  trường `web_news`):

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    - `web_news`: nguồn bài báo (vnexpress, dantri, vietnamnet)
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://vnexpress.net/chinh-phu-ban-hanh-nghi-dinh-moi-ve-gia-dat-4763835.html",
        "title": "Chính phủ ban hành nghị định mới về giá đất",
        "description": "Chính phủ hôm nay ban hành Nghị định 71, trong đó quy ...",
        "content": "Nghị định này có hiệu lực khi Luật Đất đai 2024 được thi hành ...",
        "metadata": {
            "cat": "Bất động sản",
            "subcat": "Chính sách",
            "published_date": 1719575647,
            "author": "Anh Tú"
        },
        "web_news": "vnexpress"
    },
    ```

# Các bước tiến hành

1. Chuẩn bị các thư viện, dữ liệu cần thiết
2. Xử lý dữ liệu từng site một
3. Gộp thành một file

## Chuẩn bị các thư viện cần thiết

In [1]:
import os
import json

## Chuẩn bị data cần thiết (lấy data từ mục thu thập dữ liệu)

In [2]:
DATA_FOLDER = "./data"

## Xử lý dữ liệu từng site một

Mỗi trang website chúng ta kiểm tra các trường:

- description
- author
- content
- published date

### Trang web vnexpress

In [3]:
VNEXPRESS_DATA_FOLDER = os.path.join(DATA_FOLDER, "vnexpress")
os.listdir(VNEXPRESS_DATA_FOLDER)

['bất-động-sản.json',
 'du-lịch.json',
 'giáo-dục.json',
 'giải-trí.json',
 'khoa-học.json',
 'kinh-doanh.json',
 'pháp-luật.json',
 'số-hóa.json',
 'sức-khỏe.json',
 'thế-giới.json',
 'thể-thao.json',
 'thời-sự.json',
 'xe.json',
 'đời-sống.json']

In [20]:
# Gộp các thể loại file và gộp thành 1 list
all_docs = []
for file_name in os.listdir(VNEXPRESS_DATA_FOLDER):
    with open(os.path.join(VNEXPRESS_DATA_FOLDER, file_name), encoding='utf-8') as fIn:
        current_docs = json.load(fIn)
    # web_news field
    for doc in current_docs:
        doc["web_news"] = "vnexpress"
    all_docs.extend(current_docs)

len(all_docs)

5688

In [6]:
# Kiểm tra một văn bản
all_docs[0]

{'url': 'https://vnexpress.net/vi-sao-hang-nghin-nha-tai-dinh-cu-bi-bo-hoang-4753665.html',
 'title': 'Vì sao hàng nghìn nhà tái định cư bị bỏ hoang?',
 'description': 'Cơ chế bất cập, quy hoạch thiếu thực tế, chất lượng xây dựng kém khiến hàng nghìn căn hộ tái định cư ở Hà Nội và TP HCM không có người ở chục năm qua.',
 'content': 'Gia đình 5 người nhà bà Lê Thị Liễn, 62 tuổi, đã sinh sống hàng chục năm trong căn nhà rộng khoảng 12 m2 nằm sâu trong ngõ Hàng Chiếu (quận Hoàn Kiếm, Hà Nội). Căn nhà chật hẹp được cơi nới thêm một phòng ngủ trên gác xép, sau khi con trai lấy vợ.\nNằm trong diện giãn dân khỏi phố cổ, bà Liễn cho biết gia đình không muốn chuyển sang khu tái định cư tại đường Lý Sơn (phường Thượng Thanh, quận Long Biên), bởi cách nhà cũ tới 7 km. Bà nói căn nhà ở Hàng Chiếu chật hẹp nhưng là nơi "tấc đất tấc vàng", gắn với mưu sinh.\n"Chuyển chỗ ở xa 7 km rất bất tiện và không thành viên nào muốn chuyển sang nhà tái định cư", bà Liễn nói.\nTương tự, gia đình ông Trần Hữu Sơn

#### Xử lý description

In [18]:
# Check description
all_descriptions = []
for doc in all_docs:
    all_descriptions.append(doc["description"])

Kiểm tra vài samples

In [19]:
all_descriptions[:10]

## Ta có thể thấy một vài description bị dính địa điểm ở trước

['Cơ chế bất cập, quy hoạch thiếu thực tế, chất lượng xây dựng kém khiến hàng nghìn căn hộ tái định cư ở Hà Nội và TP HCM không có người ở chục năm qua.',
 'Dự án cùng với trung tâm chính trị - hành chính và trung tâm hội nghị - biểu diễn tạo nên "trục thịnh vượng" tại trung tâm Thủy Nguyên, Hải Phòng.',
 'Mặt bằng hai trong ba khu đất để xây dự án tỷ USD ở Đông Anh cơ bản đã sạch, một khu còn nhiều đất nông nghiệp chưa giải phóng mặt bằng.',
 'Tổ hợp nhà hát, trung tâm hội nghị - tiệc cưới tại Vinhomes Ocean Park 2 sắp hoàn thành, góp phần thu hút du khách trong và ngoài nước.',
 'Lễ ra quân dự án đô thị biển Libera Nha Trang sẽ diễn ra vào chiều 23/5 tại Global City Thủ Đức với quy mô hơn 4.000 người tham gia.',
 'Tỉnh đặt mục tiêu xây nhà xã hội cao gần gấp đôi chỉ tiêu được Chính phủ giao với hơn 155.000 căn chung cư, 5.000 nhà liền kề đến 2030.',
 '6 tháng đầu năm chỉ có thêm 8 dự án nhà xã hội nên Bộ Xây dựng đánh giá năm nay khó hoàn thành chỉ tiêu 130.000 căn được giao.',
 'Dự 

In [9]:

# Some have locations before --> remove them

string_test_1 = "ĐÀ NẴNGTổ hợp nghỉ dưỡng và giải trí Cocobay từng được hứa hẹn quy mô tới 5 tỷ USD, 51 ha, với khoảng 10.000 phòng tiêu chuẩn 3-5 sao, đang phơi mưa nắng."
string_test_2 = '"Chị cả" loạt phim "Phép thuật" Shannen Doherty qua đời do ung thư còn Allysa Milano và Rose McGowan khởi xướng chiến dịch #MeToo chống quấy rối tình dục.'
string_test_3 = "UBND HN ra lệnh ..."

def remove_location_description(description_str):
    # Calculate the maximum isupper or isspace first
    count = 0
    is_last_space = False # Only cut when the last is not space
    for chac in description_str:
        if chac.isupper() or chac.isspace():
            count += 1
            if chac.isupper():
                is_last_space = False
            else:
                is_last_space = True
        else:
            if count >= 1: # Remain the first capitalized (or not word) character
                count -= 1
            break

    return description_str[count:] if not is_last_space else description_str

remove_location_description(string_test_1)

'Tổ hợp nghỉ dưỡng và giải trí Cocobay từng được hứa hẹn quy mô tới 5 tỷ USD, 51 ha, với khoảng 10.000 phòng tiêu chuẩn 3-5 sao, đang phơi mưa nắng.'

In [21]:
all_descriptions = []
changed_descriptions = []
for doc in all_docs:
    new_desc = remove_location_description(doc["description"])
    if len(new_desc) < len(doc['description']):
        changed_descriptions.append((doc["description"], new_desc))
    doc["description"] = new_desc
    all_descriptions.append(doc['description'])
changed_descriptions[:10], all_descriptions[:10]

([('BÌNH DƯƠNGDự án Charm Diamond có gần 13.000 m2 cho các tiện ích, hơn 10.000 m2 chỗ đậu xe, toàn bộ căn hộ đều có ban công với mục tiêu tối ưu trải nghiệm sống.',
   'Dự án Charm Diamond có gần 13.000 m2 cho các tiện ích, hơn 10.000 m2 chỗ đậu xe, toàn bộ căn hộ đều có ban công với mục tiêu tối ưu trải nghiệm sống.'),
  ('TP HCMCông trình mang đặc trưng của kiến trúc nhiệt đới gió mùa với hệ thống sàn được nâng cao và cấu trúc mái che đặc biệt 2 lớp chống nóng.',
   'Công trình mang đặc trưng của kiến trúc nhiệt đới gió mùa với hệ thống sàn được nâng cao và cấu trúc mái che đặc biệt 2 lớp chống nóng.'),
  ('GIA LAIKiến trúc công trình có khối cũ và mới kết nối bằng hàng hiên vừa mang dáng dấp một ngôi nhà dài, vừa có tinh thần của một ngôi nhà Bắc Bộ.',
   'Kiến trúc công trình có khối cũ và mới kết nối bằng hàng hiên vừa mang dáng dấp một ngôi nhà dài, vừa có tinh thần của một ngôi nhà Bắc Bộ.'),
  ('QUẢNG NINHSau hơn hai năm được chấp thuận chủ trương, tổ hợp du lịch nghỉ dưỡng 25

Kiểm tra bằng độ dài

In [22]:
lengths = [len(desc) for desc in all_descriptions]
max(lengths), min(lengths)

(240, 13)

In [23]:
# Kiểm tra xem có cái nào độ dài quá ngắn, quá dài không?
for doc in all_docs:
    if len(doc["description"]) < 50:
        print(doc["description"])
    if len(doc["description"]) > 220:
        print(doc["description"])

Kỹ sư cao cấp
Việt Nam và Nga hợp tác triển khai Dự án Trung tâm Nghiên cứu khoa học công nghệ hạt nhân với lò phản ứng nghiên cứu mới công suất 10 MW, mục tiêu sản xuất dược chất phóng xạ, chiếu xạ silic tạo vật liệu bán dẫn sản xuất chip.
Các nghiên cứu cấp quốc gia phục vụ mục tiêu Net Zero sẽ khuyến khích công nghệ tiên tiến như trí tuệ nhân tạo, dữ liệu lớn nhằm tối ưu quy trình sản xuất hướng tăng trưởng xanh, chuyển dịch năng lượng sạch hay ứng phó với biến đổi khí hậu.
Nhóm nghiên cứu tại Trường Đại học Công nghệ, Đại học Quốc gia Hà Nội đang phát triển công nghệ theo dõi chuyển động mắt đầu tiên tại Việt Nam có khả năng hỗ trợ sàng lọc, chẩn đoán và can thiệp cho trẻ em mắc chứng khó đọc (Dyslexia).
Tổng Bí thư Nguyễn Phú Trọng từ trần ngày 19/7/2024 do tuổi cao, bệnh nặng. VnExpress giới thiệu bài viết về Tổng Bí thư của Ủy viên Bộ Chính trị, Bí thư Trung ương Đảng, Trưởng ban Tuyên giáo Trung ương Nguyễn Trọng Nghĩa.
13h38 ngày 19/7/2024, Tổng Bí thư Nguyễn Phú Trọng từ trần

In [24]:
# Chỉ dữ văn bản đồ dài >= 50
all_docs_new = []
for doc in all_docs:
    if len(doc["description"]) >= 50:
        all_docs_new.append(doc)

all_docs = all_docs_new

In [25]:
len(all_docs)

5687

#### Xử lý author

In [26]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc["metadata"]["author"])

all_authors[:10]

['Ngọc Diễm',
 'Yên Chi',
 'Anh Tú - Ngọc Thành',
 'Văn Hà',
 'Hoài Phương',
 'Đình Trí',
 'Ngọc Diễm',
 'Yên Chi',
 'Hoài Phương',
 'Anh Kỳ']

In [28]:
# Kiểm tra tác giả có độ dài >= 50
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 50:
        print('/',doc["metadata"]["author"])

# parse errors, --> split with the first \n

/ Bình Nghi
Chủ trì thiết kế: Tô Anh Dũng
Nhóm thiết kế: Quỳnh Lưu, Thu Phương
Ảnh: Hiroyuki Oki
/ Bình Nghi
Thiết kế kiến trúc: ADAP Architects
Nguyễn Minh Phụng, Võ Đình Hiệp, Nguyễn Trung Kiên
Thi công nội thất: Nội thất NHV
Hình ảnh: Quang Trần
Nhà 75 m2 phong cách tối giản, xây trong 1,5 tháng
Kiến trúc và nội thất công trình được thiết kế theo phong cách tối giản, ưu tiên không gian mở để tiết kiệm thời gian, chi phí thực hiện. 53
/ Thu Hương
Thiết kế: Trung Trương, Trầm Nguyễn - Ltd. Studio và cộng sự
Ảnh: Hiếu T. Nguyễn
Nhà phố 64 m2 như tổ chim rợp bóng cây
Nửa diện tích ngôi nhà được thiết kế thành một khu vườn nhiều tầng bậc và một khối nhà trên cao giống như tổ chim rợp bóng cây. 18
Ngôi nhà hơn 200 m2 được bao bọc bởi rừng cây
Lớp thảm thực vật bố trí xuyên suốt từ mái, sân thượng, ban công và mặt tiền, giúp tăng mảng xanh và điều hòa nhiệt độ cho công trình. 59
Nhà cuối ngõ như 'ốc đảo xanh' nhờ thiết kế mở
Nhà cuối ngõ với 4 mặt hạn chế tầm nhìn nhưng vẫn đủ sáng, thông 

In [29]:
# Xử lý split
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 50:
        doc["metadata"]["author"] = doc["metadata"]["author"].split("\n")[0].strip()
# parse errors, --> split with the first \n

In [30]:
# Kiểm tra tác giả có độ dài >= 100 --> Unknown
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 100:
        doc["metadata"]["author"] = "Unknown"

#### Xử lý contents

In [31]:
contents = []
for doc in all_docs:
    contents.append(doc["content"])

Kiểm tra độ dài contents

In [32]:
len_contents = [len(content) for content in contents]
min(len_contents), max(len_contents)

(0, 29147)

In [40]:
# Kiểm tra tại sao lại có docs len == 0
lt = [doc['url'] for doc in all_docs if len(doc['content']) == 0]
print(*lt, sep='\n')
len(lt)
# --> Mấy MCQs báo

https://vnexpress.net/trai-nghiem-lam-sushi-nhat-ban-o-ha-noi-4736934.html
https://vnexpress.net/paris-co-lung-linh-trang-le-nhu-tren-anh-4774331.html
https://vnexpress.net/quoc-gia-chau-a-nao-co-ho-chieu-quyen-luc-nhat-the-gioi-4772397.html
https://vnexpress.net/khi-nao-nha-tho-duc-ba-paris-mo-cua-don-khach-tro-lai-4774696.html
https://vnexpress.net/nuoc-nao-co-nhieu-thanh-pho-dat-do-nhat-the-gioi-4761466.html
https://vnexpress.net/1-100-drone-ve-anh-sang-tren-troi-dem-tp-hcm-4756271.html
https://vnexpress.net/thanh-pho-nao-o-viet-nam-co-nhieu-nha-hang-sao-michelin-nhat-4766916.html
https://vnexpress.net/pho-co-la-cach-goi-khac-cua-mon-pho-dia-phuong-nao-4764381.html
https://vnexpress.net/sau-rieng-co-nguon-goc-tu-dau-4769531.html
https://vnexpress.net/noi-nao-o-can-gio-noi-tieng-voi-trai-nghiem-cau-ca-sau-4755915.html
https://vnexpress.net/duc-co-bao-nhieu-loai-xuc-xich-4758777.html
https://vnexpress.net/chao-vat-giuong-la-dac-san-cua-tinh-nao-4773801.html
https://vnexpress.net/hai-n

264

In [41]:
# Kiểm tra những docs có len < 300
lt = [doc['url'] for doc in all_docs if 0 < len(doc['content']) < 300]
print(*lt, sep='\n')
len(lt)

## Non sense, advisor, videos, mcqs ...


https://vnexpress.net/25-diem-co-the-do-truong-kinh-te-nao-4764474.html
https://vnexpress.net/hoc-cong-nghe-thong-tin-hay-khoa-hoc-du-lieu-4775854.html
https://vnexpress.net/nganh-nao-khoi-tu-nhien-hop-voi-nu-4769442.html
https://vnexpress.net/nen-hoc-dai-hoc-kinh-te-luat-hay-dai-hoc-kinh-te-tp-hcm-4774795.html
https://vnexpress.net/tiet-muc-thi-miss-grand-vietnam-nham-ba-trung-voi-ba-trieu-4775907.html
https://vnexpress.net/khi-nao-den-xi-nhan-hong-ma-khong-bi-cong-an-xu-phat-4763896.html
https://vnexpress.net/hai-tai-xe-xe-buyt-ruot-duoi-gianh-khach-tren-quoc-lo-bi-tam-giu-4766770.html
https://vnexpress.net/luat-quy-dinh-the-nao-ve-quyen-danh-ghen-khi-chong-ngoai-tinh-4763951.html
https://vnexpress.net/cong-ty-co-buoc-phai-cung-cap-nuoc-uong-cho-nhan-vien-4766042.html
https://vnexpress.net/so-do-het-han-su-dung-sau-50-nam-co-phai-tra-lai-dat-4766059.html
https://vnexpress.net/chung-cu-cu-bi-pha-do-nguoi-dan-duoc-boi-thuong-the-nao-4776710.html
https://vnexpress.net/6-trieu-dong-nen-m

44

In [43]:
all_docs = [doc for doc in all_docs if len(doc['content']) >= 300]
len(all_docs)

5379

#### Xử lý publish date

Đưa publish date về dạng chuẩn timestamp

In [44]:
# Convert all of them to timestamp
from datetime import datetime

test_string_1 = "2024-06-13T08:00:00+07:00"
test_string_2 = "2024-06-28T18:54:07+07:00"

dt = datetime.fromisoformat(test_string_2)

timestamp = dt.timestamp()
int(timestamp)

1719575647

In [45]:
# We could use GPT3.5 for converting to timestamp
def convert2timestamp(date_string):
    dt = datetime.fromisoformat(date_string)
    return int(dt.timestamp())

for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])

In [46]:
all_docs[0], len(all_docs)

({'url': 'https://vnexpress.net/vi-sao-hang-nghin-nha-tai-dinh-cu-bi-bo-hoang-4753665.html',
  'title': 'Vì sao hàng nghìn nhà tái định cư bị bỏ hoang?',
  'description': 'Cơ chế bất cập, quy hoạch thiếu thực tế, chất lượng xây dựng kém khiến hàng nghìn căn hộ tái định cư ở Hà Nội và TP HCM không có người ở chục năm qua.',
  'content': 'Gia đình 5 người nhà bà Lê Thị Liễn, 62 tuổi, đã sinh sống hàng chục năm trong căn nhà rộng khoảng 12 m2 nằm sâu trong ngõ Hàng Chiếu (quận Hoàn Kiếm, Hà Nội). Căn nhà chật hẹp được cơi nới thêm một phòng ngủ trên gác xép, sau khi con trai lấy vợ.\nNằm trong diện giãn dân khỏi phố cổ, bà Liễn cho biết gia đình không muốn chuyển sang khu tái định cư tại đường Lý Sơn (phường Thượng Thanh, quận Long Biên), bởi cách nhà cũ tới 7 km. Bà nói căn nhà ở Hàng Chiếu chật hẹp nhưng là nơi "tấc đất tấc vàng", gắn với mưu sinh.\n"Chuyển chỗ ở xa 7 km rất bất tiện và không thành viên nào muốn chuyển sang nhà tái định cư", bà Liễn nói.\nTương tự, gia đình ông Trần Hữu

#### Lưu processed data

In [47]:
with open(os.path.join(DATA_FOLDER, "vnexpress_processed.json"), "w", encoding='utf-8') as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

### Trang web dantri

Similar to trang web vnexpress

In [89]:
DANTRI_DATA_FOLDER = os.path.join(DATA_FOLDER, "dantri")
os.listdir(DANTRI_DATA_FOLDER)

['an-sinh.json',
 'bất-động-sản.json',
 'du-lịch.json',
 'giáo-dục.json',
 'giải-trí.json',
 'khoa-học---công-nghệ.json',
 'kinh-doanh.json',
 'pháp-luật.json',
 'sức-khỏe.json',
 'sức-mạnh-số.json',
 'thế-giới.json',
 'thể-thao.json',
 'tình-yêu.json',
 'việc-làm.json',
 'xe-++.json',
 'xã-hội.json',
 'đời-sống.json']

In [90]:
# Gộp các thể loại file và gộp thành 1 list
all_docs = []
for file_name in os.listdir(DANTRI_DATA_FOLDER):
    with open(os.path.join(DANTRI_DATA_FOLDER, file_name), encoding='utf-8') as fIn:
        current_docs = json.load(fIn)
    # add web_news field
    for doc in current_docs:
        doc["web_news"] = "dantri"
    all_docs.extend(current_docs)

len(all_docs)

6465

#### Xử lý description

In [91]:
#### CODE # Check description
all_descriptions = [doc['description'] for doc in all_docs]
all_descriptions[:10]

['Gần 3,4 triệu người sẽ được tăng lương hưu, trợ cấp từ 1/7 tới đây. Bảo hiểm xã hội Việt Nam đề xuất sớm có văn bản hướng dẫn về việc này để có cơ sở chi trả.',
 'Nhờ chương trình "mái ấm công đoàn", cô giáo Phạm Thị Thu Trang (ở tỉnh Thừa Thiên Huế) đã có thể thực hiện được ước mơ có căn nhà riêng cho gia đình, ổn định cuộc sống sau 12 năm lập gia đình.',
 'Mức đóng các loại hình bảo hiểm cho người lao động trên địa bàn Hà Nội được điều chỉnh tăng theo lương tối thiểu vùng, lương cơ sở từ tháng 7.',
 'Pháp luật về bảo hiểm xã hội của Việt Nam cũng như thông lệ của các nước đều không có quy định về việc hoán đổi số năm đóng bảo hiểm xã hội để về hưu trước tuổi.',
 'Công an tỉnh Đắk Lắk đã tuyển dụng vợ của 2 liệt sỹ trong vụ khủng bố tại huyện Cư Kuin và bố trí đảm nhận nhiệm vụ ở công an các xã.',
 'Chủ tịch UBND tỉnh Bình Định Phạm Anh Tuấn cho rằng, muốn giảm nghèo bền vững, phải "trao cho người dân cần câu, hướng dẫn cách làm mồi và cách câu".',
 'Một công ty ở Singapore đã khiến

#### Xử lý contents

In [92]:
len_contents = [len(doc['content']) for doc in all_docs]
min(len_contents), max(len_contents)

(0, 16961)

In [93]:
import numpy as np

len_contents = [len(doc['content']) for doc in all_docs]
mean_length = np.mean(len_contents)
std_length = np.std(len_contents)

z_score_threshold = 2
filtered_docs = []
outliers = []

for doc in all_docs:
    z_score = (len(doc['content']) - mean_length) / std_length
    if abs(z_score) <= z_score_threshold: filtered_docs.append(doc)
    else: outliers.append(doc)

len_contents = [len(doc['content']) for doc in filtered_docs]
all_docs = filtered_docs
len(filtered_docs), outliers[:10], min(len_contents), max(len_contents)

(6229,
 [{'url': 'https://dantri.com.vn/an-sinh/thu-tuong-chinh-phu-hay-song-cho-xung-dang-voi-nhung-nguoi-da-hi-sinh-20240723143428896.htm',
   'title': 'Thủ tướng Chính phủ: Hãy sống cho xứng đáng với những người đã hi sinh',
   'description': 'Thủ tướng Phạm Minh Chính nhấn mạnh, cả nước nguyện tiếp tục chăm lo chu đáo để xoa dịu nỗi đau, thấm giọt nước mắt, làm vơi đi nỗi nhớ, để đời sống vật chất, tinh thần của người có công tốt đẹp hơn.',
   'content': 'Thấu hiểu nỗi khắc khoải của người thân liệt sĩ chưa có thông tin\nPhát biểu tại Hội nghị Tri ân người có công với cách mạng năm 2024 và ra mắt Ngân hàng gen (ADN) liệt sĩ chưa xác định được thông tin và thân nhân liệt sĩ sáng 23/7, Thủ tướng Phạm Minh Chính cho biết, hội nghị hôm nay diễn ra trong niềm tiếc thương vô hạn của đồng chí, đồng bào cả nước trước sự ra đi của đồng chí Tổng Bí thư Nguyễn Phú Trọng, nhà lãnh đạo đặc biệt xuất sắc của Đảng, Nhà nước ta, người luôn đặc biệt quan tâm chăm lo công tác thương binh, liệt sĩ và

In [94]:
all_docs = [doc for doc in all_docs if len(doc['content']) >= 300]
len(all_docs)

6219

#### Xử lý author

In [95]:
#### CODE
all_authors = [doc["metadata"]["author"] for doc in all_docs]
for author in all_authors:
    if len(author) >= 50: print('/', author)
all_authors.count("Unknown"), all_authors[:10]
 

(104,
 ['Lê Thanh Xuân',
  'Vi Thảo',
  'Hoa Lê',
  'Hoa Lê',
  'Thúy Diễm',
  'Doãn Công',
  'Hạ Di',
  'Hoa Lê',
  'Hoa Lê',
  'Tùng Nguyên'])

#### Xử lý publish date

In [96]:
from datetime import datetime

def convert2timestamp(date_string):
    if isinstance(date_string, int): return date_string
    try:
        date_format = "%Y-%m-%d %H:%M"
        date_object = datetime.strptime(date_string, date_format)
        timestamp = date_object.timestamp()
        return int(timestamp)
    except: 
        return 0

date_string = "2024-07-30 17:44"
convert2timestamp(date_string)

1722336240

In [98]:
for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])
len([doc for doc in all_docs if doc['metadata']['published_date'] == 0]), all_docs[0]

(0,
 {'url': 'https://dantri.com.vn/an-sinh/lich-chi-tra-luong-huu-tro-cap-thang-7-khi-ap-dung-muc-tang-15-20240628110947647.htm',
  'title': 'Lịch chi trả lương hưu, trợ cấp tháng 7, khi áp dụng mức tăng 15%',
  'description': 'Gần 3,4 triệu người sẽ được tăng lương hưu, trợ cấp từ 1/7 tới đây. Bảo hiểm xã hội Việt Nam đề xuất sớm có văn bản hướng dẫn về việc này để có cơ sở chi trả.',
  'content': 'Lương hưu, trợ cấp sẽ tăng 15% từ 1/7 tới đây. Người đang hưởng lương hưu trước năm 1995, sau khi điều chỉnh mà có mức hưởng thấp hơn 3,2 triệu đồng/tháng thì điều chỉnh tăng 0,3 triệu đồng/tháng, có mức hưởng từ 3,2 triệu đồng/tháng đến dưới 3,5 triệu đồng/tháng thì điều chỉnh bằng 3,5 triệu đồng/tháng.\nNhư vậy, trong kỳ chi trả lương hưu, trợ cấp của tháng 7, mức hưởng của đối tượng cũng sẽ được điều chỉnh.\nVừa qua, Bảo hiểm xã hội Việt Nam đã đề xuất các cấp thẩm quyền cần sớm có văn bản hướng dẫn về dự thảo tăng lương hưu, trợ cấp 15% để đơn vị này có cơ sở chi trả lương hưu, trợ cấp

#### Lưu processed data

In [99]:
with open(os.path.join(DATA_FOLDER, "dantri_processed.json"), "w", encoding='utf-8') as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

### Trang web vietnamnet

Similar to trang web vnexpress

In [100]:
VIETNAMNET_DATA_FOLDER = os.path.join(DATA_FOLDER, "vietnamnet")
os.listdir(VIETNAMNET_DATA_FOLDER)

['bạn-đọc.json',
 'bảo-vệ-người-tiêu-dùng.json',
 'bất-động-sản.json',
 'chính-trị.json',
 'công-nghiệp-hỗ-trợ.json',
 'du-lịch.json',
 'dân-tộc---tôn-giáo.json',
 'dân-tộc-thiểu-số-và-miền-núi.json',
 'giáo-dục.json',
 'giải-trí.json',
 'kinh-doanh.json',
 'nông-thôn-mới.json',
 'nội-dung-chuyên-đề.json',
 'pháp-luật.json',
 'sức-khỏe.json',
 'thông-tin-và-truyền-thông.json',
 'thế-giới.json',
 'thể-thao.json',
 'thị-trường-tiêu-dùng.json',
 'thời-sự.json',
 'tuần-việt-nam.json',
 'văn-hóa.json',
 'ô-tô-xe-máy.json',
 'đời-sống.json']

In [102]:
# Group all the file
all_docs = []
for file_name in os.listdir(VIETNAMNET_DATA_FOLDER):
    with open(os.path.join(VIETNAMNET_DATA_FOLDER, file_name), encoding='utf-8') as fIn:
        current_docs = json.load(fIn)
    for doc in current_docs:
        doc["web_news"] = "vietnamnet"
    all_docs.extend(current_docs)

len(all_docs)

9417

#### Xử lý description

In [104]:
all_descriptions = [doc['description'] for doc in all_docs]
len_descriptions = [len(desc) for desc in all_descriptions]
min(len_descriptions), max(len_descriptions), all_descriptions[:10]

(15,
 250,
 ['Bản thân chị Hoa cũng nhiều bệnh nền nhưng không thể bỏ mặc em trai đang bên bờ sinh tử. Cạn tiền chữa bệnh cho em, có những hôm chị chỉ mua 1 cái bánh mì, ăn lai rai cả ngày cho đỡ đói.',
  'Bố bỏ đi nhiều năm, một mình mẹ làm lụng đủ nghề để nuôi 3 đứa con thơ. Trong lúc vào rừng đốt rẫy, người mẹ không may bị chết ngạt. Bạn đọc Báo VietNamNet vừa ủng hộ ba anh em mồ côi mẹ số tiền gần 202 triệu đồng.',
  'Trong dịp đầu năm 2024, Báo VietNamNet đã trao số tiền 26.735.440 đồng đến bà Nông Thị Sáu bị ung thư vú và 26.345.580 đồng đến em Chu Minh Đức bị ung thư hốc mắt.',
  'Bố mẹ và anh trai mất do tai nạn giao thông khi Hoàng Lê Đại Phúc mới hơn 5 tháng tuổi, được bà nội bế đi xin sữa. Nhờ sự quan tâm kịp thời của bạn đọc VietNamNet, cuộc sống của hai bà cháu đã dần ổn định.',
  'Ước gì chẳng phải lẻ loi/ Một người kề cạnh một người cho vui/ Ước gì nếm đủ ngọt bùi/ Để không phải cảnh ngậm ngùi cô đơn.',
  'Nhờ sức lan toả mạnh mẽ, những hoàn cảnh thương tâm được đăng tải

In [112]:
# Kiểm tra xem có cái nào độ dài quá ngắn, quá dài không?
long_docs = []
short_docs = []
for doc in all_docs:
    len_doc = len(doc["description"])
    is_short, is_long = len_doc < 50, len_doc > 220
    if is_short or is_long:
        print(doc['url'], doc['description'])
        (long_docs if is_long else short_docs).append(doc)
len(long_docs), len(short_docs)

https://vietnamnet.vn/dmk-dairy-viet-nam-tang-sua-cho-benh-nhi-o-tp-hcm-2247429.html Công ty TNHH DMK Dairy Việt Nam vừa tổ chức chương trình “Sữa yêu thương, Tết an lành”, trao tặng hàng chục ngàn hộp sữa tươi tiệt trùng Oldenburger cho các em nhỏ có hoàn cảnh khó khăn đang điều trị tại các bệnh viện nhi ở TP.HCM.
https://vietnamnet.vn/mui-tet-vuong-dau-chan-xa-2245190.html Mỗi độ xuân về, người con xa xứ không tránh khỏi chạnh lòng. Nỗi nhớ trong lòng người tha hương rất lạ: sâu lắng, dịu dàng, chôn kín. Như gái đôi mươi thầm thương trộm nhớ một ai đó, âm thầm, mãnh liệt, nồng nàn, tha thiết.
https://vietnamnet.vn/khong-co-bao-hiem-y-te-con-khanh-kiet-vi-lo-vien-phi-cho-cha-hoan-canh-kho-khan-2289039.html Trong lúc sửa mái bếp, ông Bằng không may bị trượt chân ngã xuống đất gây chấn thương sọ não nghiêm trọng. Hoàn cảnh khó khăn không có bảo hiểm y tế nên viện phí, thuốc thang đang là gánh nặng lớn với gia đình nông dân nghèo.
https://vietnamnet.vn/trao-gan-43-trieu-dong-to

(1176, 3)

In [113]:
all_docs = [doc for doc in all_docs if len(doc['description']) >= 50]
len(all_docs)

9414

#### Xử lý author

In [116]:
all_authors = [doc["metadata"]["author"] for doc in all_docs]
for author in all_authors:
    if len(author) >= 50: print('/', author)
all_authors.count(""), all_authors[:10]

/ Ủy viên Bộ Chính trị, Thủ tướng Chính phủ Phạm Minh Chính


(1113,
 ['Khánh Hòa',
  'Thiện Lương',
  'Phạm Bắc',
  'Lê Dương',
  'Nguyễn Quang Vinh',
  'Phạm Bắc',
  'Trần Tuyên',
  'Lê Nhung',
  '',
  'Trần Chánh Nghĩa'])

In [117]:
for doc in all_docs: 
    if len(doc['metadata']['author']) == 0: doc["metadata"]["author"] = "Unknown"
all_authors = [doc["metadata"]["author"] for doc in all_docs]
all_authors.count(""), all_authors[:10]

(0,
 ['Khánh Hòa',
  'Thiện Lương',
  'Phạm Bắc',
  'Lê Dương',
  'Nguyễn Quang Vinh',
  'Phạm Bắc',
  'Trần Tuyên',
  'Lê Nhung',
  'Unknown',
  'Trần Chánh Nghĩa'])

#### Xử lý contents

In [120]:
len_contents = [len(doc['content']) for doc in all_docs]
min(len_contents), max(len_contents), len_contents.count(0)

(0, 48393, 696)

In [119]:
# Kiểm tra tại sao lại có docs len == 0
for doc in all_docs:
    if len(doc["content"]) == 0: print(doc)
# CAUSE: PAGE WITH SLIGHTLY DIFFERENT STRUCTURE, HAVING div.content-detail (contains all metadata as well as content, but DON'T HAVE ANY div.main-content)

{'url': 'https://vietnamnet.vn/thu-hoi-lo-thuoc-nho-mat-tobradico-kem-chat-luong-2110120.html', 'title': 'Thu hồi lô thuốc nhỏ mắt Tobradico kém chất lượng', 'description': 'Cục Quản lý Dược vừa có công văn gửi Sở Y tế các tỉnh, thành phố thông báo về việc thu hồi thuốc nhỏ mắt Tobradico (Tobramycin (dưới dạng Tobramycin sulfat) 15mg/5ml) do không đạt tiêu chuẩn chất lượng, không an toàn cho người sử dụng.', 'content': '', 'metadata': {'cat': 'BẢO VỆ NGƯỜI TIÊU DÙNG', 'subcat': '', 'published_date': 'thứ tư, 15/02/2023 - 10:07', 'author': 'Uknown'}, 'web_news': 'vietnamnet'}
{'url': 'https://vietnamnet.vn/4-cho-can-xem-ngay-khi-mua-be-be-chon-nham-co-the-mua-phai-vo-rong-2228884.html', 'title': '4 chỗ cần xem ngay khi chọn mua bề bề', 'description': 'Bề bề (tôm tít) là món được nhiều người ưa thích, nhưng chọn bề bề thế nào để mua được những con tươi ngon thì không phải ai cũng biết.', 'content': '', 'metadata': {'cat': 'BẢO VỆ NGƯỜI TIÊU DÙNG', 'subcat': '', 'published_date': 'thứ 

In [121]:
# Kiểm tra những docs có len < 300
for doc in all_docs:
    if len(doc["content"]) > 0 and len(doc["content"]) < 300:
        print(doc)

{'url': 'https://vietnamnet.vn/ban-doc-ung-ho-cac-hoan-canh-kho-khan-10-ngay-cuoi-thang-6-2024-2298852.html', 'title': 'Bạn đọc ủng hộ các hoàn cảnh khó khăn 10 ngày cuối tháng 6/2024', 'description': 'Trong 10 ngày cuối tháng 6/2024, Báo VietNamNet đã nhận được số tiền 627.625.633 đồng giúp đỡ các hoàn cảnh khó khăn được đăng trên VietNamNet của các cá nhân và đơn vị sau.', 'content': '1. Ủng hộ tại Ngân hàng Vietcombank\n2. Ủng hộ tại Ngân hàng Vietinbank\n\nBan Bạn đọc', 'metadata': {'cat': 'BẠN ĐỌC', 'subcat': 'CHIA SẺ', 'published_date': 'chủ nhật, 07/07/2024 - 08:00', 'author': 'Thu Hiền'}, 'web_news': 'vietnamnet'}
{'url': 'https://vietnamnet.vn/ban-doc-ung-ho-cac-hoan-canh-kho-khan-20-ngay-dau-thang-2-2024-2252975.html', 'title': 'Bạn đọc ủng hộ các hoàn cảnh khó khăn 20 ngày đầu tháng 2/2024', 'description': 'Trong 20 ngày đầu tháng 2/2024, Báo VietNamNet đã nhận được số tiền 667.954.838 đồng giúp đỡ các hoàn cảnh khó khăn được đăng trên VietNamNet của các cá nhân và đơn vị sa

In [123]:
all_docs = [doc for doc in all_docs if len(doc['content']) >= 300]
len(all_docs)

8671

#### Xử lý publish date

In [124]:
from datetime import datetime
# We could use GPT3.5 for converting to timestamp
def convert2timestamp(date):
    if isinstance(date, int): return date
    try:
        date_cleaned = date.split(', ')[1]
        date_format = "%d/%m/%Y - %H:%M"
        date_object = datetime.strptime(date_cleaned, date_format)
        timestamp = date_object.timestamp()
        return int(timestamp)
    except: return 0
    
for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])

In [125]:
all_docs[0], len(all_docs)

({'url': 'https://vietnamnet.vn/can-tien-chua-benh-cho-em-nguoi-phu-nu-mua-1-cai-banh-mi-an-ca-ngay-2296216.html',
  'title': 'Cạn tiền chữa bệnh cho em, người phụ nữ cả ngày chỉ dám ăn một ổ bánh mì',
  'description': 'Bản thân chị Hoa cũng nhiều bệnh nền nhưng không thể bỏ mặc em trai đang bên bờ sinh tử. Cạn tiền chữa bệnh cho em, có những hôm chị chỉ mua 1 cái bánh mì, ăn lai rai cả ngày cho đỡ đói.',
  'content': 'Trên giường bệnh, anh Danh Nghị (SN 1975, quê Kiên Giang) vẫn nằm im lìm, thỉnh thoảng mở mắt nhưng chưa có nhận thức. Ngồi bên cạnh, chị Thị Hoa (SN 1972) nét mặt ủ dột. Chị không biết em trai còn nằm viện đến bao giờ khi mà gia đình đã cạn sạch tiền bạc, chị cũng sắp kiệt quệ sức lực.\n[Chị Hoa mỏi mệt sau nhiều ngày chăm sóc em trai nằm viện.]\nTrước đó, tháng 11/2023, anh Nghị bị ngã dẫn đến chấn thương sọ não. Từ quê Kiên Giang, anh phải chuyển lên Cần Thơ, rồi qua nhiều bệnh viện ở TP.HCM mới giữ được mạng sống.\n“Đợt đó tôi tưởng nó chết rồi, may mắn mạng lớn. Điề

#### Xử lý Category

In [127]:
for doc in all_docs:
    for field in ['cat', 'subcat']:
        doc['metadata'][field] = doc['metadata'][field].capitalize()

#### Lưu processed data

In [129]:
with open(os.path.join(DATA_FOLDER, "vietnamnet_processed.json"), "w", encoding='utf-8') as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

## Gộp dữ liệu từ một website vào thành một file

In [130]:
# Đọc từng file một
all_json_files = ["dantri_processed.json", "vietnamnet_processed.json", "vnexpress_processed.json"]
all_docs = []

for file_name in all_json_files:
    with open(os.path.join(DATA_FOLDER, file_name), encoding='utf-8') as fOut:
        all_docs.extend(json.load(fOut))

len(all_docs)


20269

In [131]:
import random
random.seed(42)
random.shuffle(all_docs)

## Lưu lại vào một cái
with open(os.path.join(DATA_FOLDER, "news_final.json"), "w", encoding='utf-8') as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)